# For colab environment

In [ ]:
# !pip install nltk transformers==4.35.0 torch==2.6.0 torchvision==0.21.0 datasets accelerate==0.24.0 huggingface==0.0.1 datasets==2.14.7

In [1]:
import torch 
print(torch.cuda.is_available())
print(torch.__version__)

True
2.6.0+cu124


In [ ]:
# !git clone https://github.com/BernardMoy/NLP-PCL-Classification.git

In [ ]:
# %cd NLP-PCL-Classification/

In [2]:
!nvidia-smi

Sat Feb 28 18:31:08 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX TITAN X     Off |   00000000:02:00.0  On |                  N/A |
| 24%   66C    P0             83W /  250W |     428MiB /  12288MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Load train and validation data set

In [3]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt
from collections import Counter

df = pd.read_csv('data/dontpatronizeme_pcl.tsv', sep='\t')

# Remove rows with NA labels 
df = df.dropna() 

# Add a bool_labels column for binary classification
df["bool_labels"] = df["label"] > 1   # is PCL if >1

# train val split 
train_labels = pd.read_csv('data/train_semeval_parids-labels.csv')["par_id"]
val_labels = pd.read_csv('data/dev_semeval_parids-labels.csv')["par_id"]
df_train = df[df["par_id"].isin(train_labels)]
df_val = df[df["par_id"].isin(val_labels)]


# Coreference resolution

In [4]:
from fastcoref import FCoref

# define the model once
model = FCoref(device='cuda:0')


def coreference_resolution(model, text):
    preds = model.predict(
        texts = [text]
    )

    clusters = preds[0].get_clusters(as_strings = False) 

    # create mappings from each pronoun indices -> entities TEXT
    d = {}
    for cluster in clusters:
        entity = cluster[0]    # IMPORTANT - The first entity is assumed to be the main entity here. Use POS tagging to further improve this. 
        refs = cluster[1::]

        for ref in refs: 
            d[ref] = text[entity[0]:entity[1]]

    # for each pronoun index (key), replace by their entity text (value) 
    sorted_keys = sorted(d.keys(), reverse = True) 
    for key in sorted_keys: 
        start, end = key 
        text = text[:start] + d[key] + text[end:]

   
    return text

   

print(coreference_resolution(model, 'We are so happy to see you using our coref package . This package is very fast !'))
print(coreference_resolution(model, 'He said the CEO of Apple was happy. Tim Cook later confirmed it .'))
print(coreference_resolution(model, 'She goes down the rabbit hole. Where Alice would discover a new reality beyond her expectations .'))
print(coreference_resolution(model, "Dr. Lester Keith , doctor and professor of business administration , and others are checking with local transportation groups to see if they can bring those in need of a meal to the college for the 4 p.m. dinner . We will also be contacting local soup kitchens as a pickup location and will work with them to transport any leftovers to them so there is no wasted food , Dr. Keith said ."))


In [ ]:
%%capture

# Apply coreference resolution to every text in the dataframe 
df_train["text_cr"] = df_train["text"].apply(lambda x: coreference_resolution(model, x))
df_val["text_cr"] = df_val["text"].apply(lambda x: coreference_resolution(model, x))

In [ ]:
df_train.head()

# Tokenization

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoConfig, Trainer, TrainingArguments

tokenizer = RobertaTokenizer.from_pretrained("roberta-base") 

# Create text with contextual information 
def tokenize(df): 
    text_with_context = df["text_cr"] # df["keyword"] + "</s><s>" + df["country_code"] + "</s><s>" + df["text"]

    encoding = tokenizer(
        text_with_context.tolist(), 
        padding="max_length",   # Add padding to shorter sentences 
        max_length=256,
        truncation = True, 
        return_attention_mask = True 
    )

    return encoding

# Convert to pyTorch dataset

In [ ]:
import torch 
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset

def to_dataset(df): 
    # Obtain tokens (input_ids, attention_mask) from the dataset 
    encoding = tokenize(df) 

    # Return huggingface dataset 
    return Dataset.from_dict({
        "input_ids": encoding["input_ids"], 
        "attention_mask": encoding["attention_mask"], 
        "label": df["bool_labels"].values 
    })

In [ ]:
train_dataset = to_dataset(df_train)
val_dataset = to_dataset(df_val) 

# set to torch format 
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Training 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate metrics 
    accuracy = accuracy_score(labels, predictions) 
    precision = precision_score(labels, predictions) 
    recall = recall_score(labels, predictions) 
    f1 = f1_score(labels, predictions) 

    return {
        "accuracy": accuracy, 
        "precision": precision, 
        "recall": recall, 
        "f1": f1 
    }


In [ ]:
# Load roberta sequence classification model 
config = AutoConfig.from_pretrained("roberta-base", num_labels=2)  # Binary classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config = config)

BATCH_SIZE = 32

# Set up training arguments 
training_args = TrainingArguments(
    fp16=True, 
    num_train_epochs=5, 
    learning_rate=2e-5, 
    weight_decay=0.01,
    warmup_steps=500, 
    save_strategy="no",  # low disk space 
    load_best_model_at_end=True, 
    logging_steps=50,
    output_dir="./predictions", 
    evaluation_strategy="no", 
    per_device_eval_batch_size=BATCH_SIZE, 
    per_device_train_batch_size=BATCH_SIZE, 
)

# Set up trainer 
trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset, 
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train() 

In [ ]:
trainer.evaluate()